<a href="https://colab.research.google.com/github/datasigntist/tensorflowCodes/blob/master/tensorflow101_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import functools 
import numpy as np 
import tensorflow as tf 
import pandas as pd

LABEL_COLUMN = "Survived"

TRAIN_DATA_URL=  "https://raw.githubusercontent.com/datasigntist/datasetsForTraining/master/train.csv" 

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL) 

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

titanicDataSet = get_dataset(train_file_path)

class extractFeaturesFromName(object):
  def __call__(self, features, labels):
    namesOfPassengers = features['Name']
    lastName_feature = tf.map_fn(lambda feat: tf.strings.strip(tf.strings.split(feat,",")[0]) ,namesOfPassengers)
    firstName_feature = tf.map_fn(lambda feat: tf.strings.strip(tf.strings.split(tf.strings.strip(tf.strings.split(feat,",")[1]),".")[1]) ,namesOfPassengers)
    title_feature  = tf.map_fn(lambda feat: tf.strings.strip(tf.strings.split(tf.strings.strip(tf.strings.split(feat,",")[1]),".")[0]) ,namesOfPassengers)
    features['last_name'] = lastName_feature
    features['first_name'] = firstName_feature
    features['title_feature'] = title_feature
    return features, labels

class replaceMissingValuesInAge(object):
  def __init__(self, meanAge):
    self.meanAge = meanAge

  def __call__(self, features, labels):
    ageOfPassengers = features['Age']
    ageOfPassengers = tf.map_fn(lambda age: (age if age>0 else self.meanAge) ,ageOfPassengers)
    features['Age'] = ageOfPassengers
    return features, labels

desc = pd.read_csv(train_file_path)["Age"].describe()
MEAN = np.float32(np.array(desc.T['mean']))

newtitanicDataSet = titanicDataSet.map(extractFeaturesFromName())
newtitanicDataSet = newtitanicDataSet.map(replaceMissingValuesInAge(MEAN))

show_batch(newtitanicDataSet)

PassengerId         : [312 811 704 378 172]
Pclass              : [1 3 3 1 3]
Name                : [b'Ryerson, Miss. Emily Borie' b'Alexander, Mr. William'
 b'Gallagher, Mr. Martin' b'Widener, Mr. Harry Elkins'
 b'Rice, Master. Arthur']
Sex                 : [b'female' b'male' b'male' b'male' b'male']
Age                 : [18. 26. 25. 27.  4.]
SibSp               : [2 0 0 0 4]
Parch               : [2 0 0 2 1]
Ticket              : [b'PC 17608' b'3474' b'36864' b'113503' b'382652']
Fare                : [262.375    7.8875   7.7417 211.5     29.125 ]
Cabin               : [b'B57 B59 B63 B66' b'' b'' b'C82' b'']
Embarked            : [b'C' b'S' b'Q' b'C' b'Q']
last_name           : [b'Ryerson' b'Alexander' b'Gallagher' b'Widener' b'Rice']
first_name          : [b'Emily Borie' b'William' b'Martin' b'Harry Elkins' b'Arthur']
title_feature       : [b'Miss' b'Mr' b'Mr' b'Mr' b'Master']
